In [1]:
%matplotlib inline

from tqdm import tqdm
import os, sys, pathlib
from pprint import pprint
import gc, time
import pickle
from importlib import reload
import logging, warnings
logging.basicConfig(level=logging.ERROR)
warnings.simplefilter("ignore")

import PyPDF2 as ppdf
import pandas as pd
import numpy as np
import sklearn
from sklearn.decomposition import PCA
import scipy.linalg as linalg
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, r2_score
from scipy.stats import wilcoxon, mannwhitneyu, linregress


import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import matplotlib.lines as mlines
from matplotlib.ticker import MaxNLocator, FormatStrFormatter
from matplotlib.collections import LineCollection

import torch

import pyaldata as pyal

if "__file__" not in dir():
    try:
        NBPath = pathlib.Path.cwd()
        RepoPath = NBPath.parent
        os.chdir(RepoPath)

        from tools import utilityTools as utility
        from tools import dataTools as dt
        from tools import lstm
        reload(lstm)
        import params
        reload(params)
        monkey_defs = params.monkey_defs
        mouse_defs = params.mouse_defs

        set_rc =  params.set_rc_params
        set_rc()
        root = params.root

    finally:
        os.chdir(NBPath)

    print('Done')

Done


In [2]:
@utility.report
def get_full_mouse_data():
    defs = mouse_defs
    
    animalList = ['mouse-data']
    animalFiles = []
    for animal in animalList:
        animalFiles.extend(utility.find_file(root / animal, 'mat'))

    AllDFs=[]
    for fname in animalFiles:
        df = dt.load_pyal_data(fname)
        df['mouse'] = fname.split(os.sep)[-1][fname.split(os.sep)[-1].find('WR'):].split('_')[0]
        df['file'] = fname.split(os.sep)[-1]
        df = defs.prep_general_mouse(df)
        AllDFs.append(df)

    allDFs_M1 = []
    for df in AllDFs:
        if 'M1_rates' in df.columns:
            allDFs_M1.append(df)


    allDFs_Str = []
    for df in AllDFs:
        if 'Str_rates' in df.columns:
            allDFs_Str.append(df)
            
    return allDFs_M1, allDFs_Str

In [3]:
allDFs_M1, allDFs_Str = get_full_mouse_data()
allDFs_M1[0].head()

/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_a

Executed: `get_full_mouse_data` in 0.8s


,trialType,timeAlign,evtAlign,rEnd,spkTimeBins,M1_spikes,Str_spikes,hTrjBfull,hTrjB,hVelB,...,mouse,file,trial_id,bin_size,idx_movement_on,idx_pull_on,idx_pull_off,target_id,M1_rates,Str_rates
0,sp,95465.0,rStart,95625,"[94475.0, 94505.0, 94535.0, 94565.0, 94595.0, ...","[[1.4142135623730951, 1.0, 0.0, 0.0, 0.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-20.241591280987624, -20.193809158354696, -2...","[[-21.617206490049753, 123.37762963545966, 9.7...","[[0.5657960870279292, 0.6693662599095281, -0.7...",...,WR40,js2p0_tbytSpkHandJsTrj10msBin_WR40_082019_ss.mat,4,0.03,33,36,38,1,"[[48.230476, 12.984568, 0.003973114, 10.430512...","[[11.545658, 0.0, 0.035090767, 0.00031967516, ..."
1,sp,103329.0,rStart,103621,"[102339.0, 102369.0, 102399.0, 102429.0, 10245...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[-22.129976070557507, -22.1091891245526, -22....","[[-22.773827685388994, 122.74653930235276, 10....","[[0.047629180458486076, 0.057504694833217705, ...",...,WR40,js2p0_tbytSpkHandJsTrj10msBin_WR40_082019_ss.mat,5,0.03,33,36,40,1,"[[2.8043926, 2.833545, 12.949476, 0.0, 0.0, 0....","[[12.949476, 0.0, 13.165841, 22.165998, 0.0, 1..."
2,sp,112122.0,rStart,112322,"[111132.0, 111162.0, 111192.0, 111222.0, 11125...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-20.088159150228577, -20.162543649753434, -2...","[[-20.51028141929519, 124.2427470581265, 10.82...","[[-0.03500524525736163, 0.07140660813481266, 0...",...,WR40,js2p0_tbytSpkHandJsTrj10msBin_WR40_082019_ss.mat,6,0.03,33,37,40,1,"[[0.034771092, 7.115508, 9.000157, 0.034771092...","[[10.21844, 0.0, 9.034929, 11.132244, 1.126516..."
3,sp,118842.0,rStart,119114,"[117852.0, 117882.0, 117912.0, 117942.0, 11797...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-22.193024870675817, -22.124509737122505, -2...","[[-22.001373239214942, 123.37312023971602, 8.6...","[[-0.03948905593039361, 0.05681994039482182, -...",...,WR40,js2p0_tbytSpkHandJsTrj10msBin_WR40_082019_ss.mat,7,0.03,33,39,42,1,"[[1.5599517, 0.003973114, 0.0, 0.0, 0.0, 0.003...","[[12.953449, 0.0, 11.132244, 12.949476, 0.0347..."
4,sp,132382.0,rStart,132918,"[131392.0, 131422.0, 131452.0, 131482.0, 13151...","[[0.0, 1.4142135623730951, 1.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-21.05636562202261, -21.05634960162285, -21....","[[-20.62071566423359, 124.04481051426757, 8.54...","[[0.08057000838697093, 0.026929242504536433, -...",...,WR40,js2p0_tbytSpkHandJsTrj10msBin_WR40_082019_ss.mat,9,0.03,33,44,51,1,"[[23.054934, 21.14687, 32.20695, 0.0, 0.0, 6.4...","[[0.00031967516, 0.0, 7.32822, 3.7476692, 0.0,..."


In [4]:
allDFs_M1, allDFs_Str = get_full_mouse_data()

rows = []
for df in allDFs_M1:
    dic = {
        'animal': df['mouse'][0],
        'areas': [x for x in list(df.columns) if '_spikes' in x],
        'trials': len(df),
        'neurons_left': df['M1_spikes'][0].shape[1] 
    }
    rows.append(dic)

summary = pd.DataFrame(rows)
summary

/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_a

Executed: `get_full_mouse_data` in 0.7s


/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/jcc319/multi_animal_alignment/packages/pyaldata/pyaldata/firing_rates.py:91: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")


,animal,areas,trials,neurons_left
0,WR40,"[M1_spikes, Str_spikes]",61,80
1,WR38,"[M1_spikes, Str_spikes]",79,95
2,WR38,"[M1_spikes, Str_spikes]",47,102
3,WR40,"[M1_spikes, Str_spikes]",86,70
4,WR39,"[M1_spikes, Str_spikes]",59,64
5,WR44,"[M1_spikes, Str_spikes]",73,55


In [5]:
summary.groupby('animal')['neurons_left'].agg(['mean', 'sem'])

,mean,sem
animal,,
WR38,98.5,3.5
WR39,64.0,NaN
WR40,75.0,5.0
WR44,55.0,NaN


In [8]:
rows = []
for df in allDFs_Str:
    dic = {        
        'animal': df['mouse'][0],
        'areas': [x for x in list(df.columns) if '_spikes' in x],
        'trials': len(df),
        'neurons_left': df['Str_spikes'][0].shape[1] 
    }
    rows.append(dic)

summary = pd.DataFrame(rows)
summary

,animal,areas,trials,neurons_left
0,WR40,"[M1_spikes, Str_spikes]",61,79
1,WR38,"[M1_spikes, Str_spikes]",79,87
2,WR38,"[M1_spikes, Str_spikes]",47,112
3,WR40,"[M1_spikes, Str_spikes]",86,69
4,WR39,"[M1_spikes, Str_spikes]",59,108
5,WR44,"[M1_spikes, Str_spikes]",73,110


In [9]:
summary.groupby('animal')['neurons_left'].agg(['mean', 'sem'])

,mean,sem
animal,,
WR38,99.5,12.5
WR39,108.0,NaN
WR40,74.0,5.0
WR44,110.0,NaN
